In [3]:
# Twitter Sentiment Investing Strategy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf
import os
plt.style.use('ggplot')

data_folder = 'C:/Users/Thomas/OneDrive/Desktop/ml-algo-trading/data'

sentiment_df = pd.read_csv(os.path.join(data_folder, 'sentiment_data.csv'))

sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])

sentiment_df = sentiment_df.set_index(['date', 'symbol'])

sentiment_df['engagement_ratio'] = sentiment_df['twitterComments']/sentiment_df['twitterLikes']

sentiment_df = sentiment_df[(sentiment_df['twitterLikes']>20)&(sentiment_df['twitterComments']>10)]

sentiment_df

twitterPosts  twitterComments  twitterLikes  \
date       symbol                                                
2021-11-18 AAPL           811.0           2592.0       21674.0   
           AMD            150.0            675.0        2949.0   
           AMZN           557.0           1315.0       12969.0   
           ATVI            82.0             36.0         131.0   
           BA              61.0             55.0         342.0   
...                         ...              ...           ...   
2023-01-04 T              184.0           3409.0       14271.0   
           TSLA          6767.0         540711.0     3810688.0   
           TSN             35.0            168.0         460.0   
           V              132.0           1008.0        5943.0   
           XOM            212.0            374.0        2071.0   

                   twitterImpressions  twitterSentiment  engagement_ratio  
date       symbol                                                          
2021-11-18 AAPL             7981808.0               NaN          0.119590  
           AMD              1645270.0               NaN          0.228891  
           AMZN             5590695.0               NaN          0.101396  
           ATVI             1310715.0               NaN          0.274809  
           BA                425847.0               NaN          0.160819  
...                               ...               ...               ...  
2023-01-04 T                 884935.0          0.590457          0.238876  
           TSLA            55464921.0          0.543057          0.141893  
           TSN                57207.0          0.561900          0.365217  
           V                 139835.0          0.567286          0.169611  
           XOM               483389.0          0.588914          0.180589  

[25375 rows x 6 columns]

In [4]:
# Aggregate Monthly and calculate average sentiment for the month

aggragated_df = (sentiment_df.reset_index('symbol').groupby([pd.Grouper(freq='M'), 'symbol'])
                    [['engagement_ratio']].mean())

aggragated_df['rank'] = (aggragated_df.groupby(level=0)['engagement_ratio']
                         .transform(lambda x: x.rank(ascending=False)))

aggragated_df

engagement_ratio  rank
date       symbol                        
2021-11-30 AAL             0.203835  37.0
           AAPL            0.256318  23.0
           ABBV            0.244677  26.0
           ABT             0.313368  14.0
           AES             0.864613   2.0
...                             ...   ...
2023-01-31 TMO             0.333313  19.0
           TSLA            0.151992  72.0
           TSN             0.280553  30.0
           V               0.194045  60.0
           XOM             0.217904  53.0

[1109 rows x 2 columns]

In [5]:
# Select Top 5 Stocks based on their cross-sectional ranking for each month

filtered_df = aggragated_df[aggragated_df['rank']<6].copy()

filtered_df = filtered_df.reset_index(level=1)

filtered_df.index = filtered_df.index+pd.DateOffset(1)

filtered_df = filtered_df.reset_index().set_index(['date', 'symbol'])

filtered_df.head(20)

engagement_ratio  rank
date       symbol                        
2021-12-01 AES             0.864613   2.0
           FCX             0.626323   5.0
           MNST            0.699721   3.0
           OXY             2.147741   1.0
           SLB             0.647374   4.0
2022-01-01 D               0.801994   5.0
           FCX             0.841220   4.0
           L               6.507246   1.0
           LUV             1.303215   2.0
           MA              0.883401   3.0
2022-02-01 AMD             0.715556   4.0
           D               1.136678   1.0
           FCX             0.655237   5.0
           LUV             1.035258   2.0
           MA              0.729063   3.0
2022-03-01 FCX             1.138332   2.0
           GILD            0.530886   5.0
           LUV             1.574377   1.0
           MRO             0.581748   4.0
           OXY             1.025579   3.0

In [6]:
# Extract the stocks to form portfolios with at the start of each new month

dates = filtered_df.index.get_level_values('date').unique().tolist()

fixed_dates = {}

for d in dates:
    
    fixed_dates[d.strftime('%Y-%m-%d')] = filtered_df.xs(d, level=0).index.tolist()
    
fixed_dates

{'2021-12-01': ['AES', 'FCX', 'MNST', 'OXY', 'SLB'],
 '2022-01-01': ['D', 'FCX', 'L', 'LUV', 'MA'],
 '2022-02-01': ['AMD', 'D', 'FCX', 'LUV', 'MA'],
 '2022-03-01': ['FCX', 'GILD', 'LUV', 'MRO', 'OXY'],
 '2022-04-01': ['A', 'CRM', 'PFE', 'PM', 'STZ'],
 '2022-05-01': ['AMD', 'CRM', 'CVX', 'J', 'KEY'],
 '2022-06-01': ['AMD', 'DD', 'FCX', 'KEY', 'LMT'],
 '2022-07-01': ['CB', 'CRM', 'DD', 'FCX', 'STZ'],
 '2022-08-01': ['A', 'DD', 'JPM', 'REGN', 'STZ'],
 '2022-09-01': ['ABT', 'DIS', 'L', 'META', 'MRNA'],
 '2022-10-01': ['J', 'KEY', 'L', 'META', 'MU'],
 '2022-11-01': ['A', 'DD', 'FCX', 'J', 'META'],
 '2022-12-01': ['AEP', 'AES', 'DD', 'J', 'STZ'],
 '2023-01-01': ['A', 'AES', 'DAL', 'J', 'KEY'],
 '2023-02-01': ['A', 'AES', 'BIIB', 'FCX', 'MDT']}

In [7]:
# Download fresh stock prices for only selected/shortlisted stocks

stocks_list = sentiment_df.index.get_level_values('symbol').unique().tolist()

prices_df = yf.download(tickers=stocks_list,
                        start='2021-01-01',
                        end='2024-02-01')

prices_df

[*********************100%%**********************]  85 of 85 completed

1 Failed download:
['ATVI']: Exception('%ticker%: No timezone found, symbol may be delisted')


Adj Close                                                         \
                     A    AAL         AAP        AAPL        ABBV         ABT   
Date                                                                            
2021-01-04  116.384750  15.13  145.845108  127.164146   92.173622  103.105186   
2021-01-05  117.336304  15.43  145.687531  128.736404   93.126717  104.380898   
2021-01-06  120.553955  15.52  154.104156  124.402924   92.322273  104.163551   
2021-01-07  123.761810  15.38  155.420410  128.647964   93.310371  105.174667   
2021-01-08  124.644676  15.13  157.635788  129.758362   93.800056  105.467606   
...                ...    ...         ...         ...         ...         ...   
2024-01-25  129.880005  15.36   66.129997  194.169998  165.130005  112.449997   
2024-01-26  130.990005  15.13   66.250000  192.419998  164.399994  112.010002   
2024-01-29  132.830002  14.94   66.959999  191.729996  163.910004  113.449997   
2024-01-30  134.279999  14.53   67.250000  188.039993  164.919998  113.480003   
2024-01-31  130.100006  14.23   66.849998  184.399994  164.399994  113.150002   

                                                          ...   Volume  \
                  AEP        AES         ALB         AMD  ...      RCL   
Date                                                      ...            
2021-01-04  73.126450  21.144056  149.119370   92.300003  ...  5732800   
2021-01-05  73.027817  21.853714  159.182465   92.769997  ...  4029500   
2021-01-06  73.610748  22.727135  171.072540   90.330002  ...  3612500   
2021-01-07  71.646706  22.499680  179.767838   95.160004  ...  3993700   
2021-01-08  71.009964  23.127451  179.865570   94.580002  ...  3104200   
...               ...        ...         ...         ...  ...      ...   
2024-01-25  78.089996  16.955368  118.449997  180.330002  ...  1909200   
2024-01-26  77.839996  16.975164  119.839996  177.250000  ...  2857900   
2024-01-29  78.379997  17.103840  123.199997  177.830002  ...  2587200   
2024-01-30  78.440002  16.796999  120.309998  172.059998  ...  2317500   
2024-01-31  78.139999  16.680000  114.739998  167.690002  ...  3840800   

                                                                               \
               REGN       SLB      STZ         T      TMO       TSLA      TSN   
Date                                                                            
2021-01-04   911100   8743400  1202600  86700816  2220400  145914600  2177400   
2021-01-05   797700  16256600  1530200  83054520  2046700   96735600  2297100   
2021-01-06  1434700  19456300  1861700  73455520  2112300  134100000  1942200   
2021-01-07  1126600  14502400  2537100  71265359  2144400  154496700  2969400   
2021-01-08  1018800  11951900  1595400  72471656  2221700  225166500  2672400   
...             ...       ...      ...       ...      ...        ...      ...   
2024-01-25   394000  11845300   797500  54457900  1748100  198076800  1551300   
2024-01-26   460500   9172000  1078500  38567300  3172300  107343200  1844500   
2024-01-29   530000  10072600  1473700  38264800  2445900  125013100  1491000   
2024-01-30   374700  47358300   950500  54946400  2301300  109982300  2063500   
2024-01-31   601900  26286300  1116300  52968100  3589400  103221400  4236300   

                                
                   V       XOM  
Date                            
2021-01-04  10318300  27764700  
2021-01-05   6869700  44035100  
2021-01-06   7206200  36484800  
2021-01-07  10967900  29528100  
2021-01-08   6513000  28628200  
...              ...       ...  
2024-01-25   6894900  22089500  
2024-01-26   8586500  20817200  
2024-01-29   6574500  18317500  
2024-01-30   6549000  19610900  
2024-01-31   6139300  22415300  

[774 rows x 510 columns]

In [9]:
# Calculate Portfolio Returns with monthly rebalancing

returns_df = np.log(prices_df['Adj Close']).diff().dropna()

portfolio_df = pd.DataFrame()

for start_date in fixed_dates.keys():
    
    end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd()).strftime('%Y-%m-%d')
    
    cols = fixed_dates[start_date]
    
    temp_df = returns_df[start_date:end_date][cols].mean(axis=1).to_frame('portfolio_return')
    
    portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)
    
portfolio_df

,portfolio_return
Date,
